In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.163  Python-3.10.11 torch-2.0.1+cpu CPU (Intel Core(TM) i5-10300H 2.50GHz)
Setup complete  (8 CPUs, 7.8 GB RAM, 303.0/453.5 GB disk)


In [ ]:
import cv2
from ultralytics import YOLO
from twilio.rest import Client

# Initialize YOLO model
model = YOLO("best.pt", conf=0.8)
names = model.names

# Twilio API credentials
account_sid = 'twilio account sid'
auth_token = 'twilio auth token'
client = Client(account_sid, auth_token)

# Function to send SMS
def send_sms():
    message = client.messages.create(
        from_='twilio account number',
        body='Animal Detected: monkey or pig',
        to='your phone number'
    )
    print("SMS sent:", message.sid)

# Capture video for 5 seconds
video_capture = cv2.VideoCapture(0)
start_time = cv2.getTickCount()
consecutive_detections = 0
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Perform object detection on the frame
    results = model.predict(frame, imgsz=512)
    for r in results:
        for c in r.boxes.cls:
            detected_class = names[int(c)]
            print("Detected:", detected_class)
            if detected_class in ['monkey', 'pig']:
                consecutive_detections += 1
                if consecutive_detections >= 5:
                    send_sms()
                    consecutive_detections = 0
            else:
                consecutive_detections = 0

    # Check if 5 seconds have elapsed
    current_time = cv2.getTickCount()
    elapsed_time = (current_time - start_time) / cv2.getTickFrequency()
    if elapsed_time >= 5:
        break

# Release video capture and close windows
video_capture.release()
cv2.destroyAllWindows()


In [ ]:
import socket
import cv2
from ultralytics import YOLO
from twilio.rest import Client

# Initialize YOLO model
model = YOLO("best.pt", conf=0.8)
names = model.names

# Twilio API credentials
account_sid = 'twilio account sid'
auth_token = 'twilio auth token'
twilio_client = Client(account_sid, auth_token)

# Function to send SMS
def send_sms():
    message = twilio_client.messages.create(
        from_='twilio account number',
        body='Animal Detected: monkey or pig',
        to='your phone number'
    )
    print("SMS sent:", message.sid)

# Function to perform object detection
def perform_object_detection():
    # Capture video for 5 seconds
    video_capture = cv2.VideoCapture(0)
    start_time = cv2.getTickCount()
    consecutive_detections = 0
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Perform object detection on the frame
        results = model.predict(frame, imgsz=512)
        for r in results:
            for c in r.boxes.cls:
                detected_class = names[int(c)]
                print("Detected:", detected_class)
                if detected_class in ['monkey', 'pig']:
                    consecutive_detections += 1
                    if consecutive_detections >= 5:
                        send_sms()
                        consecutive_detections = 0
                else:
                    consecutive_detections = 0

        # Check if 5 seconds have elapsed
        current_time = cv2.getTickCount()
        elapsed_time = (current_time - start_time) / cv2.getTickFrequency()
        if elapsed_time >= 5:
            break

    # Release video capture and close windows
    video_capture.release()
    cv2.destroyAllWindows()

# Server code to listen for messages
HOST = '192.168.29.107'
PORT = 12345

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print("Server is listening for incoming connections...")
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            data = conn.recv(1024).decode()
            if not data:
                break
            print(f"Received message: {data}")
            if data == 'Motion Detected!':
                perform_object_detection()
